In [1]:
import pandas as pd
import numpy as np


import matplotlib.pyplot as plt
plt.style.use ('ggplot')
import seaborn as sns
%matplotlib inline
sns.set_style(style="whitegrid")

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import NMF
from sklearn.feature_extraction import text

from bs4 import BeautifulSoup

import nltk, string, contractions

from gensim.corpora.dictionary import Dictionary
from gensim.models.nmf import Nmf
from gensim.models.coherencemodel import CoherenceModel

from operator import itemgetter

In [2]:
df = pd.read_csv('data/modeling_ready_microwave1')

In [3]:
df.head()

,customer_id,review_id,product_id,star_rating,helpful_votes,total_votes,verified_purchase,review_headline,review_body,review_date,review_wordcount,clean_review
0,44300577,R2ZU11YALTJNZX,B0009KMYHI,1.0,0.0,0.0,Y,I have had this microwave for just over 3 year...,I have had this microwave for just over 3 year...,2015-08-31,91,I have had this microwave for just over 3 year...
1,50952586,RVBITZNBVJ8AI,B0009KMYHI,5.0,0.0,0.0,Y,It takes a good bit of room on the counter,Man this thing cooks evenly and fast! It takes...,2015-08-30,23,Man this thing cooks evenly and fast! It takes...
2,31144708,R44LZL0OR5EWP,B0009KMYDM,5.0,0.0,0.0,Y,Five Stars,Works great! Have had it over a year and no su...,2015-08-30,17,Works great! Have had it over a year and no su...
3,14667788,RPWWYNUD9X64U,B0009KMYHI,5.0,0.0,0.0,Y,... received this several years ago and it sti...,I ordered and received this several years ago ...,2015-08-29,13,I ordered and received this several years ago ...
4,25468208,RS76DCCGM685B,B0009KMYGY,1.0,3.0,3.0,Y,Avoid Panasonic - Doors Don't Close,I have had this microwave for just over 2 year...,2015-08-28,131,I have had this microwave for just over 2 year...


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1875 entries, 0 to 1874
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   customer_id        1875 non-null   int64         
 1   review_id          1875 non-null   object        
 2   product_id         1875 non-null   object        
 3   star_rating        1875 non-null   float64       
 4   helpful_votes      1875 non-null   float64       
 5   total_votes        1875 non-null   float64       
 6   verified_purchase  1875 non-null   object        
 7   review_headline    1875 non-null   object        
 8   review_body        1875 non-null   object        
 9   review_date        1875 non-null   datetime64[ns]
 10  review_wordcount   1875 non-null   int64         
 11  clean_review       1875 non-null   object        
dtypes: datetime64[ns](1), float64(3), int64(2), object(6)
memory usage: 175.9+ KB


In [4]:
df.review_date = pd.to_datetime(df.review_date)

In [5]:
df.describe()

,customer_id,star_rating,helpful_votes,total_votes,review_wordcount
count,1.875000e+03,1875.000000,1875.000000,1875.000000,1875.000000
mean,3.209401e+07,3.850133,3.107733,3.628267,80.606933
std,1.476223e+07,1.496151,20.062047,20.926332,97.269705
min,8.722000e+04,1.000000,0.000000,0.000000,1.000000
25%,1.748780e+07,3.000000,0.000000,0.000000,28.000000
50%,3.303278e+07,5.000000,1.000000,1.000000,51.000000
75%,4.624525e+07,5.000000,2.000000,2.000000,94.000000
max,5.309426e+07,5.000000,589.000000,599.000000,1152.000000


In [7]:
punct =[]
punct += list(string.punctuation)
punct += '’'
punct.remove("'")
def remove_punctuations(text):
    for punctuation in punct:
        text = text.replace(punctuation, ' ')
    return text

In [8]:
def nlp_prep(df):
    # lowercase everything
    # get rid of '\n' from whitespace
    # regex remove hyperlinks
    # remove punctuation
    # remove ' s ' from removing punctuation
    
    # lowercase everything
    df['model_ready'] = df['clean_review'].apply(lambda x: x.lower())
    # expand contractions
    df['model_ready'] = df['model_ready'].apply(lambda x:[contractions.fix(word) for word in x.split()])
    df['model_ready'] = [' '.join(map(str, l)) for l in df['model_ready']]
    # get rid of '\n' from whitespace 
    df['model_ready'] = df['model_ready'].apply(lambda x: x.replace('\n', ' '))
    # regex remove hyperlinks
    df['model_ready'] = df['model_ready'].str.replace('http\S+|www.\S+', '', case=False)
    # remove punctuations
    df['model_ready'] = df['model_ready'].apply(remove_punctuations)
    # remove ' s ' that was created after removing punctuations
    df['model_ready'] = df['model_ready'].apply(lambda x: str(x).replace(" s ", " "))
    return df

In [9]:
nlp_prep(df)

,customer_id,review_id,product_id,star_rating,helpful_votes,total_votes,verified_purchase,review_headline,review_body,review_date,review_wordcount,clean_review,model_ready
0,44300577,R2ZU11YALTJNZX,B0009KMYHI,1.0,0.0,0.0,Y,I have had this microwave for just over 3 year...,I have had this microwave for just over 3 year...,2015-08-31,91,I have had this microwave for just over 3 year...,i have had this microwave for just over 3 year...
1,50952586,RVBITZNBVJ8AI,B0009KMYHI,5.0,0.0,0.0,Y,It takes a good bit of room on the counter,Man this thing cooks evenly and fast! It takes...,2015-08-30,23,Man this thing cooks evenly and fast! It takes...,man this thing cooks evenly and fast it takes...
2,31144708,R44LZL0OR5EWP,B0009KMYDM,5.0,0.0,0.0,Y,Five Stars,Works great! Have had it over a year and no su...,2015-08-30,17,Works great! Have had it over a year and no su...,works great have had it over a year and no su...
3,14667788,RPWWYNUD9X64U,B0009KMYHI,5.0,0.0,0.0,Y,... received this several years ago and it sti...,I ordered and received this several years ago ...,2015-08-29,13,I ordered and received this several years ago ...,i ordered and received this several years ago ...
4,25468208,RS76DCCGM685B,B0009KMYGY,1.0,3.0,3.0,Y,Avoid Panasonic - Doors Don't Close,I have had this microwave for just over 2 year...,2015-08-28,131,I have had this microwave for just over 2 year...,i have had this microwave for just over 2 year...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1870,33705501,R34GY91QSWWUE4,B0009KMYGY,4.0,60.0,64.0,N,Just right for me...,The Panasonic NN-H765WF was exactly what I was...,2006-03-23,74,The Panasonic NN-H765WF was exactly what I was...,the panasonic nn h765wf was exactly what i was...
1871,39268146,R7N1LRCV8GU6L,B0009KMYHI,5.0,5.0,10.0,Y,good looking-good cooking,this microwave cooks food very evenly-no need ...,2006-03-19,21,this microwave cooks food very evenly-no need ...,this microwave cooks food very evenly no need ...
1872,19314267,R15KWS9QSI7URC,B0009KMYDM,5.0,12.0,13.0,Y,Panasonic NN-H965WF Luxury full-size 2.2 cu.ft...,"its a great microwave , i use it every day , i...",2006-03-16,67,"its a great microwave , i use it every day , i...",its a great microwave i use it every day i...
1873,11400155,RK5HSUM1GNRH8,B0009KMYDM,5.0,4.0,9.0,N,Panasonic Microwave Oven NN-965WF,Have had it for only a short time but it has p...,2006-02-24,20,Have had it for only a short time but it has p...,have had it for only a short time but it has p...


In [10]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.WordNetLemmatizer()

def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]

In [11]:
keep_words = ['not', 'ain', 'aren', "aren't", 'couldn', "couldn't", 
              'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 
              'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 
              'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn',
              "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 
              'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 
              'wouldn', "wouldn't", 'fire', 'off']

In [154]:
test_stop_words = ['great', 'excellent', '1', '2', 'feature', 'nice', 'old']
review_stop_words = ['panasonic', 'really', 'husband', 'thanks', 'thank', 'ha', 
              'just', 'thing', 'did', 'nn', 'wa', 'yr', 'u', 'say', 'doe',
              'mom', 'christmas', 'gift', 'got', 'way', 'le', 'daughter',
              'e','not','love','good','bought','great microwave','micro',
              'great oven','microwave','product','work great','nice work',
              'work great use','work great love','feature work great',
              'unit work great','oven work great','easy use love','old oven',
              'old old','20 year old','unit', 'not', 'work', 'amazon', 'com',
              'old', 'wife', 'highly', 'recommend', 'like', 'charm']
for _ in text.ENGLISH_STOP_WORDS:
    if _ in keep_words:
        pass
    else:
        review_stop_words.append(_)
for _ in test_stop_words:
    review_stop_words.append(_)

In [51]:
len(review_stop_words)

372

In [74]:
vectorizer = TfidfVectorizer(tokenizer = lemmatize_text,
                             stop_words= review_stop_words,
                             max_features = 5000,
                             min_df = 3,
                             max_df =  .85,
                             ngram_range=(0, 2)
                            )

In [75]:
X = df['model_ready']

In [76]:
X = vectorizer.fit_transform(X)

/home/eric/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['20', 'easy', 'oven', 'use', 'year'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [55]:
X.shape

(1875, 5000)

In [56]:
df.head()

,customer_id,review_id,product_id,star_rating,helpful_votes,total_votes,verified_purchase,review_headline,review_body,review_date,review_wordcount,clean_review,model_ready,token
0,44300577,R2ZU11YALTJNZX,B0009KMYHI,1.0,0.0,0.0,Y,I have had this microwave for just over 3 year...,I have had this microwave for just over 3 year...,2015-08-31,91,I have had this microwave for just over 3 year...,i have had this microwave for just over 3 year...,"[i, have, had, this, microwave, for, just, ove..."
1,50952586,RVBITZNBVJ8AI,B0009KMYHI,5.0,0.0,0.0,Y,It takes a good bit of room on the counter,Man this thing cooks evenly and fast! It takes...,2015-08-30,23,Man this thing cooks evenly and fast! It takes...,man this thing cooks evenly and fast it takes...,"[man, this, thing, cooks, evenly, and, fast, i..."
2,31144708,R44LZL0OR5EWP,B0009KMYDM,5.0,0.0,0.0,Y,Five Stars,Works great! Have had it over a year and no su...,2015-08-30,17,Works great! Have had it over a year and no su...,works great have had it over a year and no su...,"[works, great, have, had, it, over, a, year, a..."
3,14667788,RPWWYNUD9X64U,B0009KMYHI,5.0,0.0,0.0,Y,... received this several years ago and it sti...,I ordered and received this several years ago ...,2015-08-29,13,I ordered and received this several years ago ...,i ordered and received this several years ago ...,"[i, ordered, and, received, this, several, yea..."
4,25468208,RS76DCCGM685B,B0009KMYGY,1.0,3.0,3.0,Y,Avoid Panasonic - Doors Don't Close,I have had this microwave for just over 2 year...,2015-08-28,131,I have had this microwave for just over 2 year...,i have had this microwave for just over 2 year...,"[i, have, had, this, microwave, for, just, ove..."


In [77]:
nmf = NMF(
    n_components=5,
    init='nndsvd'
).fit(X)

nmf_output = nmf.fit_transform(X)

In [21]:
def show_topics(vectorizer=vectorizer, lda_model=nmf, n_words=20):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

In [22]:
topic_keywords = show_topics(vectorizer=vectorizer, lda_model=nmf, n_words=10)        

In [23]:
def topic_featuring(n_components=10, n_words=10):
    
    nmf = NMF(
        n_components=n_components,
        init='nndsvd'
    ).fit(X)

    nmf_output = nmf.fit_transform(X)

    topic_keywords = show_topics(vectorizer=vectorizer, lda_model=nmf, n_words=n_words)

    # Topic - Keywords Dataframe
    df_topic_keywords = pd.DataFrame(topic_keywords)
    df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
    df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]

    Topics_theme = range(n_components)
    df_topic_keywords['topic_theme'] = Topics_theme
    df_topic_keywords.set_index('topic_theme', inplace=True)
    return df_topic_keywords.T

In [78]:
topic_featuring(5, 10)

topic_theme,0,1,2,3,4
Word 0,cook,year,easy,door,oven
Word 1,food,lasted,use,open,watt
Word 2,sensor,month,easy use,latch,1250
Word 3,like,model,pleased,close,power
Word 4,power,new,recommend,button,1250 watt
Word 5,time,died,highly,push,large
Word 6,reheat,working,powerful,door latch,time
Word 7,button,warranty,highly recommend,door open,inverter
Word 8,cooking,service,powerful easy,hard,new oven
Word 9,popcorn,buy,clean,open door,cooking


In [61]:
show_topics(vectorizer=vectorizer, lda_model=nmf, n_words=10)

[array(['time', 'long time', 'long', 'run', 'time used', 'used',
        'cooking time', 'time time', 'cook time', 'sure'], dtype='<U22'),
 array(['door', 'close', 'slam', 'hard', 'door close', 'shut', 'slam door',
        'start', 'hard close', 'turn'], dtype='<U22'),
 array(['easy use', 'easy', 'use', 'powerful easy', 'working easy',
        'clean', 'suggest', 'cook easy', 'use cook', 'use easy'],
       dtype='<U22'),
 array(['open', 'door', 'door open', 'open door', 'push', 'hand', 'pull',
        'push open', 'button open', 'open push'], dtype='<U22'),
 array(['oven', 'new oven', 'recommend oven', 'oven worked', 'large oven',
        'owned', 'year oven', 'oven use', 'oven cook', 'oven fit'],
       dtype='<U22'),
 array(['fit', 'space', 'counter', 'counter space', 'kitchen', 'needed',
        'cabinet', 'fit space', 'look', 'huge'], dtype='<U22'),
 array(['sensor', 'reheat', 'sensor reheat', 'sensor cook',
        'reheat sensor', 'setting', 'defrost', 'reheat function', 'warm',

In [62]:
df['token'] = df['model_ready'].apply(nltk.word_tokenize)

In [108]:
def topic_modeler(max_features=5000, min_df=2, max_df=.85, ngram_max = 2, topic_max = 20, topic_step = 1):
    vectorizer = TfidfVectorizer(tokenizer = lemmatize_text,
                             stop_words= review_stop_words,
                             max_features = max_features,
                             min_df = min_df,
                             max_df =  max_df,
                             ngram_range=(0, ngram_max)
                            )
    X = df['model_ready']
    X = vectorizer.fit_transform(X)
    # Use Gensim's NMF to get the best num of topics via coherence score
    texts = df['token']

    # Create a dictionary
    # In gensim a dictionary is a mapping between words and their integer id
    dictionary = Dictionary(texts)

    # Filter out extremes to limit the number of features
    dictionary.filter_extremes(
        no_below=min_df,
        no_above=max_df,
        keep_n=max_features
    )

    # Create the bag-of-words format (list of (token_id, token_count))
    corpus = [dictionary.doc2bow(text) for text in texts]

    # Create a list of the topic numbers we want to try
    topic_nums = list(np.arange(1, topic_max + 1, topic_step))

    # Run the nmf model and calculate the coherence score
    # for each number of topics
    coherence_scores = []

    for num in topic_nums:
        gs_nmf = Nmf(
            corpus=corpus,
            num_topics=num,
            id2word=dictionary,
            chunksize=2000,
            passes=5,
            kappa=.1,
            minimum_probability=0.01,
            w_max_iter=300,
            w_stop_condition=0.0001,
            h_max_iter=100,
            h_stop_condition=0.001,
            eval_every=10,
            normalize=True,
            random_state=42
        )

        # Run the coherence model to get the score
        cm = CoherenceModel(
            model=gs_nmf,
            texts=texts,
            dictionary=dictionary,
            coherence='c_v'
        )

        coherence_scores.append(round(cm.get_coherence(), 5))

    # Get the number of topics with the highest coherence score
    scores = list(zip(topic_nums, coherence_scores))
    best_num_topics = sorted(scores, key=itemgetter(1), reverse=True)[0][0]
    
#     if len(coherence_scores) == topic_max:
    fig, ax = plt.subplots(figsize=(10,6))
    sns.lineplot(x=range(0,20), y = coherence_scores)
    plt.show()

In [ ]:
topic_modeler(10000, 5)

/home/eric/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['20', 'easy', 'oven', 'use', 'year'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [155]:
def nmf_featurizer(max_features=10000, min_df=3, max_df=.85, ngram_max = 2, n_components=5, n_words=10):
    vectorizer = TfidfVectorizer(tokenizer = lemmatize_text,
                             stop_words= review_stop_words,
                             max_features = max_features,
                             min_df = min_df,
                             max_df =  max_df,
                             ngram_range=(0, ngram_max)
                            )
    X = df['model_ready']
    X = vectorizer.fit_transform(X)
    
    nmf = NMF(
        n_components=n_components,
        init='nndsvd'
    ).fit(X)

    nmf_output = nmf.fit_transform(X)

    topic_keywords = show_topics(vectorizer=vectorizer, lda_model=nmf, n_words=n_words)

    # Topic - Keywords Dataframe
    df_topic_keywords = pd.DataFrame(topic_keywords)
    df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
    df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]

    Topics_theme = range(n_components)
    df_topic_keywords['topic_theme'] = Topics_theme
    df_topic_keywords.set_index('topic_theme', inplace=True)
    return df_topic_keywords.T

In [156]:
nmf_featurizer(10000,3,.85,3,17,12)

/home/eric/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['20', 'easy', 'oven', 'use', 'year'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


topic_theme,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
Word 0,time,year,easy,door,oven,fit,sensor,month,price,power,cook,popcorn,watt,item,big,fine,powerful
Word 1,cooking,lasted,use,open,new oven,large,reheat,working,best,level,food,button,1250,lot,happy,worked,loud
Word 2,used,model,easy use,latch,owned,space,sensor reheat,service,store,power level,fast,setting,1250 watt,issue,expected,worked fine,powerful large
Word 3,second,new,pleased,close,large oven,size,food,buy,buy,inverter,heat,bag,watt power,time item,working,far,big powerful
Word 4,minute,lasted year,clean,door latch,oven worked,counter,defrost,repair,right,power setting,evenly,popcorn button,1250 watt power,hot,turkey,broke,food
Word 5,long,died,easy operate,push,year oven,needed,function,warranty,size,setting,cook food,popcorn setting,little,advertised,thought,working fine,large
Word 6,cooking time,10,powerful easy,door open,inverter,kitchen,reheat function,customer,needed,lower,cook fast,pop,watt oven,purchased,inside,control,powerful easy
Word 7,use,10 year,powerful easy use,button,oven use,need,turbo,stopped,new,set,quickly,burn,1000,better,big inside,use fine,powerful easy use
Word 8,long time,replace,easy clean,hard,cooking,counter space,button,center,arrived,off,heat food,make,1000 watt,arrived,lot,fix,make
Word 9,day,20,operate,open door,oven cook,look,sensor cook,day,pleased,lot,cook evenly,size,cubic,defrost,big cook,far working,sure


In [157]:
nmf_featurizer(10000,2,.85,3,17,12)

/home/eric/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['20', 'easy', 'oven', 'use', 'year'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


topic_theme,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
Word 0,time,service,easy,door,oven,large,sensor,model,price,power,year,cook,watt,working,big,powerful,popcorn
Word 1,item,repair,easy use,open,new oven,fit,reheat,new,best,level,lasted,food,1250,month,happy,loud,button
Word 2,used,warranty,use,latch,owned,space,sensor reheat,replace,store,power level,lasted year,heat,1250 watt,buy,expected,powerful large,setting
Word 3,cooking,customer,clean,close,large oven,size,food,previous,right,inverter,died,evenly,watt power,quit,big cook,powerful easy use,bag
Word 4,long,day,pleased,door latch,oven worked,counter,defrost,new model,size,power setting,20,fast,1250 watt power,6 month,turkey,powerful easy,popcorn button
Word 5,cooking time,customer service,easy operate,push,cooking,kitchen,function,older,arrived,setting,10,cook food,little,6,working,large,pop
Word 6,minute,center,powerful easy,door open,problem,need,reheat function,year,happy,lower,10 year,cook fast,watt oven,quit working,big need,big powerful,popcorn setting
Word 7,use,called,powerful easy use,hard,year oven,counter space,sensor cook,similar,delivery,off,20 year,heat food,1000,stopped,big inside,powerful cook,burn
Word 8,second,shipping,operate,button,inverter,needed,turbo,older model,value,set,worked,quickly,1000 watt,stopped working,inside,make,minute
Word 9,long time,week,easy clean,open door,oven cook,look,sensor reheat function,replaced,buy,cooking,second,happy,1250 watt oven,far,perfect,sure,make


In [158]:
nmf_featurizer(10000,4,.85,3,17,12)

/home/eric/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['20', 'easy', 'oven', 'use', 'year'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


topic_theme,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
Word 0,time,year,easy,door,oven,fit,sensor,service,price,power,model,cook,watt,popcorn,powerful,food,month
Word 1,used,lasted,use,open,new oven,large,reheat,repair,store,level,new,fast,1250,button,large,heat,buy
Word 2,cooking,lasted year,easy use,latch,owned,space,sensor reheat,customer,best,power level,replace,cook fast,1250 watt,setting,loud,evenly,working
Word 3,item,died,pleased,close,large oven,big,defrost,day,right,inverter,previous,sensor cook,watt power,bag,big powerful,heat food,worked
Word 4,long,10,clean,door latch,oven worked,size,function,warranty,size,power setting,new model,potato,little,popcorn button,big,happy,6 month
Word 5,cooking time,20,easy operate,push,cooking,counter,food,customer service,needed,setting,older,cook food,1250 watt power,pop,powerful easy,quickly,quit
Word 6,use,10 year,operate,door open,inverter,kitchen,reheat function,center,expected,set,similar,cook evenly,watt oven,popcorn setting,powerful easy use,cook food,6
Word 7,second,20 year,easy clean,hard,problem,need,turbo,called,arrived,lower,year,evenly,1000,push,sure,heating,quit working
Word 8,lot,second,plenty,open door,year oven,needed,sensor reheat function,shipping,pleased,off,replaced,minute,1000 watt,burn,make,food evenly,far
Word 9,long time,3,powerful easy,problem,oven use,counter space,warm,week,better,cooking,pleased,larger,1200 watt,minute,spacious,purchase,money


/home/eric/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['20', 'easy', 'oven', 'use', 'year'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


topic_theme,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
Word 0,time,model,easy,door,oven,fit,sensor,month,power,year,like,cook,price,recommend,watt,popcorn,powerful
Word 1,cooking,new,use,open,new oven,big,reheat,working,level,lasted,far,fast,store,highly,1250,button,loud
Word 2,item,replace,easy use,latch,owned,large,sensor reheat,service,power level,lasted year,little,heat,best,highly recommend,1250 watt,setting,big
Word 3,used,previous,clean,close,recommend oven,space,food,repair,inverter,died,like charm,evenly,right,pleased,watt power,bag,powerful large
Word 4,minute,new model,pleased,door latch,large oven,counter,defrost,warranty,power setting,20,charm,food,buy,larger,1250 watt power,popcorn button,big powerful
Word 5,long,older,easy operate,push,oven worked,size,function,buy,setting,10,second,cook fast,size,use highly,little,pop,food
Word 6,cooking time,year,powerful easy,door open,year oven,kitchen,reheat function,customer,lower,10 year,used,cook food,expected,use highly recommend,watt oven,popcorn setting,powerful easy use
Word 7,use,replaced,powerful easy use,button,cooking,need,button,day,off,20 year,look,quickly,needed,looking,1000,burn,powerful easy
Word 8,second,similar,working,hard,oven use,counter space,hot,week,set,worked,like look,happy,arrived,amazing,1000 watt,size,spacious
Word 9,long time,pleased,easy clean,open door,problem,needed,sensor cook,stopped,lot power,second,used like,heat food,better,performance,cubic,make,sure


In [ ]:
cooking time, new model, usability, door latch, oven, 
external size, programmed functions, customer service, power, long lifetime
satisfaction (like), cooking time, SOUP, recommend / match, wattage
popcorn, power

like, charm

In [113]:
nmf_featurizer(10000,2,.95,3,17, 12)

/home/eric/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['20', 'easy', 'oven', 'use', 'year'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


topic_theme,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
Word 0,time,door,easy,open,oven,large,sensor,month,price,power,like,heat,year,watt,recommend,popcorn,big
Word 1,item,latch,use,button,new oven,fit,reheat,service,best,level,far,food,lasted,1250,highly,setting,powerful
Word 2,cooking,door latch,easy use,door,recommend oven,space,sensor reheat,working,store,power level,little,cook,model,1250 watt,highly recommend,bag,working
Word 3,cook,close,clean,push,owned,counter,food,repair,right,inverter,second,evenly,lasted year,watt power,pleased,button,big powerful
Word 4,used,shut,powerful easy,door open,oven worked,size,function,warranty,buy,power setting,like charm,fast,died,1250 watt power,looking,popcorn setting,loud
Word 5,minute,problem,powerful easy use,open door,large oven,kitchen,defrost,buy,arrived,setting,charm,happy,new,little,larger,pop,tray
Word 6,cooking time,slam,easy operate,push button,new,need,reheat function,customer,size,lower,used,heat food,10,watt oven,amazing,popcorn button,promised
Word 7,long,door close,pleased,button open,cooking,counter space,cook,center,new,off,look,quickly,10 year,1000,use highly recommend,burn,big cook
Word 8,use,broke,operate,hand,problem,wanted,sensor cook,week,expected,set,used like,cook food,20,1000 watt,use highly,size,happy
Word 9,start,hard,easy clean,pull,year oven,needed,button,day,needed,cooking,like look,cook fast,replace,1250 watt oven,lot,popping,turkey


In [153]:
nmf_featurizer(15000,2,.95,3,17, 12)

/home/eric/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['20', 'easy', 'oven', 'use', 'year'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


topic_theme,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
Word 0,time,door,easy,open,oven,big,sensor,month,price,power,like,heat,year,watt,popcorn,recommend,powerful
Word 1,item,latch,use,button,new oven,fit,reheat,working,best,level,far,food,lasted,1250,setting,highly,loud
Word 2,cooking,door latch,easy use,door,recommend oven,large,sensor reheat,service,store,power level,second,cook,model,1250 watt,bag,highly recommend,powerful large
Word 3,cook,close,clean,push,owned,space,food,repair,right,inverter,little,evenly,lasted year,watt power,button,pleased,powerful easy use
Word 4,used,shut,easy operate,door open,large oven,counter,function,warranty,buy,power setting,charm,fast,died,1250 watt power,popcorn setting,looking,powerful easy
Word 5,minute,problem,powerful easy,open door,cooking,size,defrost,buy,arrived,setting,like charm,happy,new,little,pop,larger,big powerful
Word 6,long,slam,powerful easy use,push button,oven worked,kitchen,reheat function,customer,size,lower,used,heat food,10,watt oven,popcorn button,amazing,large
Word 7,cooking time,door close,pleased,button open,problem,need,cook,week,needed,off,look,quickly,10 year,1000,burn,use highly recommend,big
Word 8,use,broke,operate,hand,new,counter space,sensor cook,center,new,set,used like,cook food,20,1000 watt,size,use highly,powerful cook
Word 9,start,hard,easy clean,pull,year oven,wanted,button,day,delivery,cooking,like look,cook fast,replace,size,popping,size,promised


### Goals

- Coherence scoring
- topic selection / tuning
- product feature scoring (topics with coherence score)
- 